# Become a Master Strategist with OR-Tools 
> Discover linear optimization in Python

- toc: false 
- badges: true
- comments: true
- categories: [linear optimization, OR-Tools, MIP, python, tutorial]
- image: images/ortools/ortools_thumbnail.png
- permalink: /ortools/

<center><div id="anim_container"><img alt="Illustration of linear optimization" src="https://mlabonne.github.io/blog/images/ortools/units.png" loading="lazy" id="anim_inner"></div></center>

In real warfare and strategy games, strategists are faced with a common problem: how to **allocate their resources to be the most efficient possible**? Let's take an example, we have three resources: 🌾**food**, 🪵**wood**, and 🪙**gold**. We can produce three types of units: 🗡️**swordsmen**, 🏹**bowmen**, and 🐎**horsemen**. 🐎Horsemen are **better than 🏹bowmen**, who are in turn **better than 🗡️swordsmen**. The following table provides the **cost and power of each unit** (they're freely inspired by a real strategy game, Age of Empires IV):
<br/>
<br/>

| Unit | 🌾Food | 🪵Wood | 🪙Gold | 💪Power |
| :--- | :---: | :---: | :---: | :---: |
| 🗡️Swordsman | 60 | 20 | 0 | 70 |
| 🏹Bowman | 80 | 10 | 40 | 95 |
| 🐎Horseman | 140 | 0 |100 | 230 |

<br/>

Imagine that we have 1200 🌾**food**, 800 🪵**wood**, and 600 🪙**gold**. How to **maximize the power of our army** considering these resources? We could simply find the unit with the **best power/cost ratio**, **take as many of them as possible**, and repeat the process with the other two units. But this "guess and check" solution **might not even be optimal**. Now imagine we have **millions of units and resources and other constraints**: this greedy strategy is very likely to **completely miss the optimal solution**. It is possible to **use a machine learning** (ML) algorithm (a metaheuristic like a genetic algorithm) to solve this problem, but we have **no guarantee that the solution will be optimal either**.

Fortunately for us, there is a method that can **solve our problem in an optimal way**: linear optimization (or linear programming), which is part of the field of [operations research](https://en.wikipedia.org/wiki/Operations_research) (OR). In this article, we're going to **find the best solution possible to this problem**. We'll use OR-Tools, a **powerful Python library** specialized in optimization. It will help us to **model this problem** in linear optimization terms and **output an optimal solution** with a solver. Finally, we will write a **model and a function** that can **take on a bigger challenge** and actually **solve a whole class of optimization problems**.

## 🥇 I. Finding the optimal solution

Okay, let's play with linear optimization in Python. There are **different libraries** such as the multi-purposed `SciPy`, the beginner-friendly `PuLP`, the exhaustive `pyomo`, and many others. Today, we are going to use [Google OR-Tools](https://developers.google.com/optimization), which is quite **user-friendly**, comes with several **prepackaged solvers**, and has by far the **most stars** on [GitHub](https://github.com/google/or-tools). If the installation doesn't work, please **restart the kernel and try again**: it can fail sometimes. ¯\\\_(ツ)\_/¯

In [1]:
!python -m pip install --upgrade --user -q ortools

All these libraries have a **hidden benefit**: they act as **interfaces** to **use the same model with different solvers**. Solvers like [Gurobi](https://www.gurobi.com/), [Cplex](https://www.ibm.com/analytics/cplex-optimizer), or [SCIP](https://www.scipopt.org/) have their own APIs, but the models they create are **linked to a specific solver**. OR-Tools allows us to use an abstract and quite pythonic way of **modeling our problems**, and then choose **one or several solvers** to find an optimal solution.

<center class="dark-bg">
    <video autoplay loop muted width="800" class="no-margin">
        <source src="https://mlabonne.github.io/blog/images/ortools/ortools_solvers.mp4" type="video/mp4"></source>
    </video>
</center>

OR-Tools comes with its **own linear programming solver**, called **GLOP** for Google Linear Optimization Package. It is an open source project created by Google's Operations Research Team and written in C++. According to Google, it is used to [stabilize YouTube videos](https://ai.googleblog.com/2014/09/sudoku-linear-optimization-and-ten-cent.html) and in various projects such a [cooperative multi-agent reinforcement learning architecture](https://proceedings.neurips.cc/paper/2019/hash/3c3c139bd8467c1587a41081ad78045e-Abstract.html). Other solvers are available, such as **SCIP**, an excellent **non-commercial solver** created in 2005 and **updated and maintained** to this day. We could use other options like Gurobi and Cplex but we would need to **install them on top of OR-Tools** and get the **appropriate licenses** (which can be quite costly). For now, let's try GLOP.

In [2]:
# Import OR-Tools' wrapper for linear solvers
from ortools.linear_solver import pywraplp

# Create a linear solver using the GLOP backend
solver = pywraplp.Solver('Maximize army power', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

We created an **instance** of the OR-Tools solver using GLOP. The first thing we want to define are the **variables we want to optimize**. In our example, we have three variables: the number of 🗡️**swordsmen**, 🏹**bowmen**, and 🐎**horsemen** in the army. OR-Tools accepts three types of variables:

* [`NumVar`](https://google.github.io/or-tools/python/ortools/linear_solver/pywraplp.html#Solver.NumVar) for **continuous variables**;
* [`IntVar`](https://google.github.io/or-tools/python/ortools/linear_solver/pywraplp.html#Solver.IntVar) for **integer variables**;
* [`BoolVar`](https://google.github.io/or-tools/python/ortools/linear_solver/pywraplp.html#Solver.BoolVar) for **boolean variables**.

We're looking for **round numbers of units**, so let's choose `IntVar`. We then need to specify **lower and upper bounds** for these variables. We want at least 0 unit, but we **don't really have an upper bound**. So we can argue that our **upper bound is infinity** and `solver.infinity()` instead. Actually $\infty$ might be a **little too much considering the number of resources we have**. We could also take $1000$ as an upper bound for instance, since **we can't go that high anyway**. But let's keep $∞$ for now.

Let's call the number of **🗡️swordsmen** $swordsmen$, the number of **🏹bowmen** $bowmen$, and the number of **🐎horsemen** $horsemen$. We can formally write:

$$0 \leq swordsmen < \infty \\ 0 \leq bowmen < \infty \\ 0 \leq horsemen < \infty$$

Good, let's translate it into code. The syntax is **quite straightforward** with OR-Tools.

In [3]:
# Create the variables we want to optimize
swordsmen = solver.IntVar(0, solver.infinity(), 'swordsmen')
bowmen = solver.IntVar(0, solver.infinity(), 'bowmen')
horsemen = solver.IntVar(0, solver.infinity(), 'horsemen')

We defined our variables, but another essential parameter are the **constraints**. In our case, we have a **limited number of resources** we can use to **produce units**. In other words, **we can't spend more resources than we have**: for instance, the 🌾**food** spent to recruit units **cannot be higher than 1200**. The same is true with 🪵**wood** (800) and  🪙**gold** (600).

According to our table, units have the following costs:

* **1 swordsman** = 🌾60 + 🪵20;
* **1 bowman** = 🌾80 + 🪵10 + 🪙40;
* **1 horseman** = 🌾140 + 🪙100.

We can write these constraints as follows:

$$60\times swordsmen + 80\times bowmen + 140\times horsemen \leq 1200 \\
20\times swordsmen + 10\times bowmen \leq 800 \\
40\times bowmen + 100\times horsemen \leq 600$$

In OR-Tools, we simply add the constraints to our solver instance with [`solver.Add()`](https://google.github.io/or-tools/python/ortools/linear_solver/pywraplp.html#Solver.Add).

In [4]:
# Add constraints for each resource
solver.Add(swordsmen*60 + bowmen*80 + horsemen*140 <= 1200) # Food
solver.Add(swordsmen*20 + bowmen*10 <= 800) # Wood
solver.Add(bowmen*40 + horsemen*100 <= 600) # Gold

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7f68c2688c90> >

Now that we have our variables and constraints, we want to **define our goal** (or **objective function**). In linear optimization, this function **has to be linear**, so of the form $f(x, y, z) = ax + by + cz + d$. In our example, the objective is quite clear: we want to **recruit the army with the highest power**. The table gives us the following power values:

* **1 swordsman** = 💪70;
* **1 bowman** = 💪95;
* **1 horseman** = 💪230.

Maximizing the power of the army amounts to **maximizing the sum of the power of each unit**. Let's call $units$ the set of our three parameters $\{swordsmen, bowmen, horsemen\}$. Our objective function can be written as:

$$max \ f(units) = 70\times swordsmen + 95\times bowmen + 230\times horsemen$$

In general, there are two types of objective functions: **minimizing** and **maximizing**. In OR-Tools, we declare this goal with [`solver.Maximize()`](https://google.github.io/or-tools/python/ortools/linear_solver/pywraplp.html#Solver.Maximize) or [`solver.Minimize()`](https://google.github.io/or-tools/python/ortools/linear_solver/pywraplp.html#Solver.Minimize).

In [5]:
# Maximize the objective function
solver.Maximize(swordsmen*70 + bowmen*95 + horsemen*230)

And we're done! There are **three steps to model an optimization problem**:

1. **Declaring the variables to optimize** with lower and upper bounds;
2. **Adding constraints** on these variables;
3. **Defining the objective function** to maximize or to minimize.

Now that is clear, we can ask the solver to find an optimal solution for us. This is done with [`solver.Solve()`](https://google.github.io/or-tools/python/ortools/linear_solver/pywraplp.html#Solver.Solve) and returns a status. This status can then be used **to check if the solution is indeed optimal**. Let's print the **highest total power** we can get with the **best army configuration**.

In [6]:
# Solve problem
status = solver.Solve()

# If an optimal solution has been found, print results
if status == pywraplp.Solver.OPTIMAL:
  print('================= Solution =================')
  print(f'Solved in {solver.wall_time():.2f} milliseconds in {solver.iterations()} iterations')
  print()
  print(f'Optimal power = {solver.Objective().Value()} 💪power')
  print('Army:')
  print(f' - 🗡️Swordsmen = {swordsmen.solution_value()}')
  print(f' - 🏹Bowmen = {bowmen.solution_value()}')
  print(f' - 🐎Horsemen = {horsemen.solution_value()}')
else:
  print('The solver could not find an optimal solution.')

================= Solution =================
Solved in 87.00 milliseconds in 2 iterations

Optimal power = 1800.0 💪power
Army:
 - 🗡️Swordsmen = 6.0000000000000036
 - 🏹Bowmen = 0.0
 - 🐎Horsemen = 5.999999999999999


Great! The solver found an optimal solution: our army has a **total power of 💪1800 with 6 🗡️swordsmen and 6 🐎horsemen** (sorry bowmen!).

Let's unpack this result: the solver decided to take the **maximum number of 🐎horsemen** (6, since we only have 🪙600 and they each cost 🪙100). The remaining resources are **spent in 🗡️swordsmen**: we have $1200 - 6 \times 140 = 360$ 🌾food left, which is why the solver **chose 6 🗡️swordsmen**. We can deduce that the **horsemen are the best unit** and the **bowmen are the worst one** because they haven't been chosen at all.

Okay, but there's something quite weird: these numbers are **not round**, even though we specified that we wanted **integers**. So what happened? This strange behavior is due to the GLOP solver: it is a **linear programming** (LP) solver, and not an **integer linear programming** (ILP) one. In summary, it can **only use real numbers and not integers** as variables. So why did we **declare our variables as integers** if it doesn't take it into account?

GLOP cannot solve ILP problems, but other solvers can. Actually, a lot of them are **mixed integer linear programming** (MILP, commonly called MIP) solvers. This means that they can consider both **continuous** (real numbers) and **discrete** (integers) variables. A particular case of discrete values are **boolean variables** to represent decisions with 0-1 values.

We talked about **SCIP** earlier, which is a good example since **it can solve both MILP and MINLP** (mixed integer **nonlinear** programming) problems. Another potential candidate is [CBC](https://projects.coin-or.org/Cbc), an open source **MIP solver directly available through OR-Tools**. Thanks to library, we can **use the same model and just change the solver** to SCIP or CBC.

In [7]:
# Create the linear solver using the CBC backend
solver = pywraplp.Solver('Maximize army power', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# 1. Create the variables we want to optimize
swordsmen = solver.IntVar(0, solver.infinity(), 'swordsmen')
bowmen = solver.IntVar(0, solver.infinity(), 'bowmen')
horsemen = solver.IntVar(0, solver.infinity(), 'horsemen')

# 2. Add constraints for each resource
solver.Add(swordsmen*60 + bowmen*80 + horsemen*140 <= 1200)
solver.Add(swordsmen*20 + bowmen*10 <= 800)
solver.Add(bowmen*40 + horsemen*100 <= 600)

# 3. Maximize the objective function
solver.Maximize(swordsmen*70 + bowmen*95 + horsemen*230)

# Solve problem
status = solver.Solve()

# If an optimal solution has been found, print results
if status == pywraplp.Solver.OPTIMAL:
  print('================= Solution =================')
  print(f'Solved in {solver.wall_time():.2f} milliseconds in {solver.iterations()} iterations')
  print()
  print(f'Optimal value = {solver.Objective().Value()} 💪power')
  print('Army:')
  print(f' - 🗡️Swordsmen = {swordsmen.solution_value()}')
  print(f' - 🏹Bowmen = {bowmen.solution_value()}')
  print(f' - 🐎Horsemen = {horsemen.solution_value()}')
else:
  print('The solver could not find an optimal solution.')

================= Solution =================
Solved in 3.00 milliseconds in 0 iterations

Optimal value = 1800.0 💪power
Army:
 - 🗡️Swordsmen = 6.0
 - 🏹Bowmen = 0.0
 - 🐎Horsemen = 6.0


Strictly speaking, our variables are still floats (`type(swordsmen.solution_value()) = float`) but we can see that **they don't have weird decimals anymore**: the CBC solver really considered them as **integers**. In general, we can just **round up these values** since the error is insignificant, but it is important to remember to choose the appropriate solver according to the **studied problem**: **LP** (continuous variables) or **MIP** (combination of continuous and discrete variables). There are other types such as **quadratic** (QP) or **nonlinear** (NLP or MINLP, with an **exponential objective function or constraints** for instance).

<center><div id="anim_container"><img alt="LP, MIP, etc." src="https://mlabonne.github.io/blog/images/ortools/continuousvsdiscrete.png" loading="lazy" id="anim_inner"></div></center>

## 🧱 III. Building a general model

But what if our **resources change**? Or if the cost of a unit is **not the same**? What if we upgraded horsemen and **their power increased**? One of the **best perks** of OR-Tools is that it uses a **general-purpose programming language** like Python. Instead of **static numbers**, we can store our parameters in **objects like dictionaries or arrays**. The code won't be **as readable**, but it becomes **much more flexible**: actually, it can be so flexible than we can **solve an entire class of optimization problems without changing the model** (just the parameters).

Let's transform our **input parameters into arrays** and feed them to the solver **through a function**.

In [8]:
# Inputs
UNITS = ['🗡️Swordsmen', '🏹Bowmen', '🐎Horsemen']

DATA = [[60, 20, 0, 70],
        [80, 10, 40, 95],
        [140, 0, 100, 230]]

RESOURCES = [1200, 800, 600]


def solve_army(UNITS, DATA, RESOURCES):
  # Create the linear solver using the CBC backend
  solver = pywraplp.Solver('Maximize army power', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

  # 1. Create the variables we want to optimize
  units = [solver.IntVar(0, solver.infinity(), unit) for unit in UNITS]

  # 2. Add constraints for each resource
  for r, _ in enumerate(RESOURCES):
    solver.Add(sum(DATA[u][r] * units[u] for u, _ in enumerate(units)) <= RESOURCES[r])

  # 3. Maximize the objective function
  solver.Maximize(sum(DATA[u][-1] * units[u] for u, _ in enumerate(units)))

  # Solve problem
  status = solver.Solve()

  # If an optimal solution has been found, print results
  if status == pywraplp.Solver.OPTIMAL:
    print('================= Solution =================')
    print(f'Solved in {solver.wall_time():.2f} milliseconds in {solver.iterations()} iterations')
    print()
    print(f'Optimal value = {solver.Objective().Value()} 💪power')
    print('Army:')
    for u, _ in enumerate(units):
      print(f' - {units[u].name()} = {units[u].solution_value()}')
  else:
    print('The solver could not find an optimal solution.')

solve_army(UNITS, DATA, RESOURCES)

================= Solution =================
Solved in 2.00 milliseconds in 0 iterations

Optimal value = 1800.0 💪power
Army:
 - 🗡️Swordsmen = 6.0
 - 🏹Bowmen = 0.0
 - 🐎Horsemen = 6.0


Good, the results are the same: our code seems to work. Now let's **change the parameters** to tackle a slightly more complex problem. 

Imagine we have a **lot more resources**: 🌾183000, 🪵90512, and 🪙80150, so we can also **produce a lot more units**! This is the new table:

| Unit | 🌾Food | 🪵Wood | 🪙Gold | 💪Attack | ❤️Health
| :--- | :---: | :---: | :---: | :---: | :---: |
| 🗡️Swordsman      | 60  | 20  | 0   | 6   | 70 |
| 🛡️Man-at-arms    | 100 | 0   | 20  | 12  | 155 |
| 🏹Bowman         | 30  | 50  | 0   | 5   | 70 |
| ❌Crossbowman    | 80  | 0   | 40  | 12  | 80 |
| 🔫Handcannoneer  | 120 | 0   | 120 | 35  | 150 |
| 🐎Horseman       | 100 | 20  | 0   | 9   | 125 |
| ♞Knight          | 140 | 0   | 100 | 24  | 230 | 
| 🐏Battering ram  | 0   | 300 | 0   | 200 | 700 |
| 🎯Springald      | 0   | 250 | 250 | 30 | 200 |

Notice that we transformed the 💪**power** into two values: 💪**attack** and ❤️ **health**, which is closer to reality. Health values **are higher than attack values**, which is why we want to **add a weight to make them more comparable**. Let's take 10 as an example, so $power = 10 \times attack + health$. We call the set of variables to optimize $units = \{swordsmen,  \dots, springalds\}$, where every element is the number of the corresponding unit. Our objective function becomes:

$$max \ f(units) = \sum_{u \in units} (10\times power + health) \cdot u$$

Adapting our code to this new problem is actually quite simple: we just have to **change the input parameters** and update the **objective function**.

In [9]:
UNITS = [
    '🗡️Swordsmen',
    '🛡️Men-at-arms',
    '🏹Bowmen',
    '❌Crossbowmen',
    '🔫Handcannoneers',
    '🐎Horsemen',
    '♞Knights',
    '🐏Battering rams',
    '🎯Springalds',
    '🪨Mangonels',
]

DATA = [
    [60, 20, 0, 6, 70],
    [100, 0, 20, 12, 155],
    [30, 50, 0, 5, 70],
    [80, 0, 40, 12, 80],
    [120, 0, 120, 35, 150],
    [100, 20, 0, 9, 125],
    [140, 0, 100, 24, 230],
    [0, 300, 0, 200, 700],
    [0, 250, 250, 30, 200],
    [0, 400, 200, 12*3, 240]
]

RESOURCES = [183000, 90512, 80150]


def solve_army(UNITS, DATA, RESOURCES):
  # Create the linear solver using the CBC backend
  solver = pywraplp.Solver('Maximize army power', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

  # 1. Create the variables we want to optimize
  units = [solver.IntVar(0, solver.infinity(), unit) for unit in UNITS]

  # 2. Add constraints for each resource
  for r, _ in enumerate(RESOURCES):
    solver.Add(sum(DATA[u][r] * units[u] for u, _ in enumerate(units)) <= RESOURCES[r])

  # 3. Maximize the new objective function
  solver.Maximize(sum((10*DATA[u][-2] + DATA[u][-1]) * units[u] for u, _ in enumerate(units)))

  # Solve problem
  status = solver.Solve()

  # If an optimal solution has been found, print results
  if status == pywraplp.Solver.OPTIMAL:
    print('================= Solution =================')
    print(f'Solved in {solver.wall_time():.2f} milliseconds in {solver.iterations()} iterations')
    print()
    print(f'Optimal value = {solver.Objective().Value()} 💪power')
    print('Army:')
    for u, _ in enumerate(units):
      print(f' - {units[u].name()} = {units[u].solution_value()}')
  else:
    print('The solver could not find an optimal solution.')

solve_army(UNITS, DATA, RESOURCES)

================= Solution =================
Solved in 74.00 milliseconds in 412 iterations

Optimal value = 1393145.0 💪power
Army:
 - 🗡️Swordsmen = 2.0
 - 🛡️Men-at-arms = 1283.0
 - 🏹Bowmen = 3.0
 - ❌Crossbowmen = 0.0
 - 🔫Handcannoneers = 454.0
 - 🐎Horsemen = 0.0
 - ♞Knights = 0.0
 - 🐏Battering rams = 301.0
 - 🎯Springalds = 0.0
 - 🪨Mangonels = 0.0


This problem would **take a long time for humans to address**, but the ILP solver did it **in the blink of an eye**. Better than that: it also gives us the **guarantee that our solution is optimal**, which is always nice to know! We could **increase the number of units**, give **billions of resources** but you get the picture: it would **just take longer to obtain a solution**, but it **wouldn't change the problem**.

## ⚔️ IV. Time for war

Now, let's say we scouted our enemy and know that the **enemy army has a 💪power of 1,000,000**. We could build a **much better army**, but our resources are precious and it **wouldn't be very efficient**: all we have to do is to build an army with a 💪power **higher than 1,000,000** (even 1,000,001 would be enough). In other words, it means we're looking for the **most cost-efficient unit** and we want the **minimum number** of them to defeat the enemy army. Knowing the **number of units to produce** will also tell us **how much resource we need**.

We can reuse our **input parameters** since they didn't change. But the constraint is different this time: we want a **💪power higher than 1,000,000**. It means that the **sum of the power of the chosen units** must be higher than this number.

$$\sum_{u \in units} (10\times attack + health) \cdot u > 1\,000\,000$$

In code, we can **loop through our units and resources** to design this constraint.

The **objective function** also has to change. Our goal is to **minimize the sum of resources spent to build the army**.

$$min \ f(units) = \sum_{u \in units} (food + wood + gold) \cdot u$$

Once again, we can loop **through our resources** to implement it in OR-Tools.

In [10]:
def solve_army(UNITS, DATA, RESOURCES):
  # Create the linear solver using the CBC backend
  solver = pywraplp.Solver('Minimize resource consumption', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

  # 1. Create the variables we want to optimize
  units = [solver.IntVar(0, solver.infinity(), unit) for unit in UNITS]

  # 2. Add constraints for each resource
  for r, _ in enumerate(RESOURCES):
    solver.Add(sum((10 * DATA[u][-2] + DATA[u][-1]) * units[u] for u, _ in enumerate(units)) >= 1000001)

  # 3. Minimize the objective function
  solver.Minimize(sum((DATA[u][0] + DATA[u][1] + DATA[u][2]) * units[u] for u, _ in enumerate(units)))

  # Solve problem
  status = solver.Solve()

  # If an optimal solution has been found, print results
  if status == pywraplp.Solver.OPTIMAL:
    print('================= Solution =================')
    print(f'Solved in {solver.wall_time():.2f} milliseconds in {solver.iterations()} iterations')
    print()

    power = sum((10 * DATA[u][-2] + DATA[u][-1]) * units[u].solution_value() for u, _ in enumerate(units))
    print(f'Optimal value = {solver.Objective().Value()} 🌾🪵🪙resources')
    print(f'Power = 💪{power}')
    print('Army:')
    for u, _ in enumerate(units):
      print(f' - {units[u].name()} = {units[u].solution_value()}')
    print()

    food = sum((DATA[u][0]) * units[u].solution_value() for u, _ in enumerate(units))
    wood = sum((DATA[u][1]) * units[u].solution_value() for u, _ in enumerate(units))
    gold = sum((DATA[u][2]) * units[u].solution_value() for u, _ in enumerate(units))
    print('Resources:')
    print(f' - 🌾Food = {food}')
    print(f' - 🪵Wood = {wood}')
    print(f' - 🪙Gold = {gold}')
  else:
      print('The solver could not find an optimal solution.')

solve_army(UNITS, DATA, RESOURCES)

================= Solution =================
Solved in 4.00 milliseconds in 0 iterations

Optimal value = 111300.0 🌾🪵🪙resources
Power = 💪1001700.0
Army:
 - 🗡️Swordsmen = 0.0
 - 🛡️Men-at-arms = 0.0
 - 🏹Bowmen = 0.0
 - ❌Crossbowmen = 0.0
 - 🔫Handcannoneers = 0.0
 - 🐎Horsemen = 0.0
 - ♞Knights = 0.0
 - 🐏Battering rams = 371.0
 - 🎯Springalds = 0.0
 - 🪨Mangonels = 0.0

Resources:
 - 🌾Food = 0.0
 - 🪵Wood = 111300.0
 - 🪙Gold = 0.0


The solver found an optimal solution: we need to **build 371 🐏battering rams for a total cost of 111,300 🪵wood**. Wait, what if **we don't have that much wood**? In the previous section, we only had 🌾183000, 🪵90512, and 🪙80150: we cannot produce 371 🐏battering rams. 😱

So is it possible to **take these limited resources into account** and still try to build the **cheapest better army**? Actually, it's super easy: we just have to **copy/paste the constraints from the previous section**.

In [11]:
def solve_army(UNITS, DATA, RESOURCES):
  # Create the linear solver using the CBC backend
  solver = pywraplp.Solver('Minimize resource consumption', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

  # 1. Create the variables we want to optimize
  units = [solver.IntVar(0, solver.infinity(), unit) for unit in UNITS]

  # 2. Add constraints for each resource
  for r, _ in enumerate(RESOURCES):
    solver.Add(sum((10 * DATA[u][-2] + DATA[u][-1]) * units[u] for u, _ in enumerate(units)) >= 1000001)

  # Old constraints for limited resources
  for r, _ in enumerate(RESOURCES):
    solver.Add(sum(DATA[u][r] * units[u] for u, _ in enumerate(units)) <= RESOURCES[r])

  # 3. Minimize the objective function
  solver.Minimize(sum((DATA[u][0] + DATA[u][1] + DATA[u][2]) * units[u] for u, _ in enumerate(units)))

  # Solve problem
  status = solver.Solve()

  # If an optimal solution has been found, print results
  if status == pywraplp.Solver.OPTIMAL:
    print('================= Solution =================')
    print(f'Solved in {solver.wall_time():.2f} milliseconds in {solver.iterations()} iterations')
    print()

    power = sum((10 * DATA[u][-2] + DATA[u][-1]) * units[u].solution_value() for u, _ in enumerate(units))
    print(f'Optimal value = {solver.Objective().Value()} 🌾🪵🪙resources')
    print(f'Power = 💪{power}')
    print('Army:')
    for u, _ in enumerate(units):
      print(f' - {units[u].name()} = {units[u].solution_value()}')
    print()
    
    food = sum((DATA[u][0]) * units[u].solution_value() for u, _ in enumerate(units))
    wood = sum((DATA[u][1]) * units[u].solution_value() for u, _ in enumerate(units))
    gold = sum((DATA[u][2]) * units[u].solution_value() for u, _ in enumerate(units))
    print('Resources:')
    print(f' - 🌾Food = {food}')
    print(f' - 🪵Wood = {wood}')
    print(f' - 🪙Gold = {gold}')
  else:
      print('The solver could not find an optimal solution.')

solve_army(UNITS, DATA, RESOURCES)

================= Solution =================
Solved in 28.00 milliseconds in 1 iterations

Optimal value = 172100.0 🌾🪵🪙resources
Power = 💪1000105.0
Army:
 - 🗡️Swordsmen = 1.0
 - 🛡️Men-at-arms = 681.0
 - 🏹Bowmen = 0.0
 - ❌Crossbowmen = 0.0
 - 🔫Handcannoneers = 0.0
 - 🐎Horsemen = 0.0
 - ♞Knights = 0.0
 - 🐏Battering rams = 301.0
 - 🎯Springalds = 0.0
 - 🪨Mangonels = 0.0

Resources:
 - 🌾Food = 68160.0
 - 🪵Wood = 90320.0
 - 🪙Gold = 13620.0


Since we now have a limited resource of 🪵wood, the number of 🐏battering rams **sadly dropped from 371 to 301**. In exchange, we got **681 🛡️men-at-arms and 1 lost 🗡️swordsman** (welcome to them). The total cost of the army is of **172,100**, which is **much higher** than the 111,300 we previously found (+65% increase) but it **truly is the optimal solution under these constraints**. It shows that **we should produce more wood** because these 🐏battering rams are extremely cost-efficient!

## 🔚 V. Conclusion

Optimization is often **neglected in favor of machine learning techniques**, but both have their merits: linear optimization can produce an **optimal solution in a undetermined amount of time**, while machine learning can **approximate complex functions in no time**. There is no training in LP, but an **expert is required to build a mathematical model**. Machine learning needs data, but the models can be **used as black boxes** to solve a problem. As a rule of thumb, problems that **do not have a particular time constraint and/or are not extremely complex** can be advantageously solved with **linear programming**.

In this article,

* We learned about **interfaces for optimization**, and especially about Google **OR-Tools**;
* We talked about **solvers and types of optimization problems**: LP, MIP, NLP;
* We modeled and solved an **extremely common optimization problem** in a **optimal way** and **generalized our model** through a function;
* We **reframed this problem** and merged two sets of constraints to obtain **the best configuration for the lowest price**.

There are a lot more problems where optimization can be applied: for instance, how to **create school timetables that satisfy everybody's requirements**? How to **deliver 1,000 different orders in a minimum amount of time**? Where to **create a new metro line to maximize its usefulness**? In future articles, we'll talk about **new types of applications** for these techniques, including satisfiability and nonliner problems.

I hope you enjoyed this article! Feel free to **share it and spread the knowledge about linear optimization**. Don't forget to **[follow me on Twitter](https://twitter.com/maximelabonne)** where I post summaries of these articles. Cheers!